In [ ]:
import os
from langchain_groq import ChatGroq

# Set your Groq API key
os.environ["GROQ_API_KEY"] = ""  # <-- REPLACE WITH YOUR KEY

# Initialize Groq LLM (e.g., mixtral, llama3-70b, gemma-7b)
llm = ChatGroq(
    model="llama3-70b-8192",  # or "llama3-70b-8192", "gemma-7b-it"
    temperature=0.1,
    request_timeout=60
)



In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load your PDF
loader = PyPDFLoader("mypdf_customer.pdf")  # <-- CHANGE THIS
pages = loader.load()

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(pages)

print(f"Loaded {len(pages)} pages and split into {len(chunks)} chunks.")


Loaded 30 pages and split into 97 chunks.


In [3]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant

# Embedding model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Qdrant client (localhost)
client = QdrantClient(host="localhost", port=6333)

# Recreate collection
collection_name = "customer_support_docs"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

# Upload to Qdrant
qdrant = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embedding,
)

qdrant.add_documents(chunks)


C:\Users\Sibghatullah\AppData\Local\Temp\ipykernel_16008\4199777356.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Sibghatullah\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Sibghatullah\AppData\Local\Temp\ipykernel_16008\4199777356.py:14: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the futu

['c71b9388460345a68f23822383096c2b',
 '1d016760836649afa962c5402094883f',
 'ef1eb8fca2964751b144c74a2b9b9b83',
 '96988be36a8b47ecb2f709b44bb3ad88',
 '43547324da404d028e19327829bdee2c',
 '1eca9f50b36247f78c7e2fb86ebaef17',
 '6df70e29f1a44933ac679a2ac5bede67',
 '3d6d6f1c7df947f997873c1b656c79dd',
 'd59e18c63b834699ada1d73dbd5d5270',
 'b66dade49f2f44f68216f56bbca1e10e',
 '77f0d2a1e90d479d8cafee6dbfad0126',
 'af513e04197a46998038b407dacdce61',
 '8abe9d79a90040fea7722d4d980a24af',
 '86ebfc5002c0418da5fb153652748a59',
 '8563aca3d41846cfbcf265c6dfa80356',
 '8077a7039cea4e06b3d28f8395e2fe52',
 '42bc4c2721c8461daff2b0bbab8ececb',
 '65a8858d7d9c47f18961883f79078de6',
 'bcf7ffe7df1944548101e355955a7717',
 '00bfab03e4a24997b7fd0a12baa45e33',
 'b2fc9b08358e47fd96807f67fb640f71',
 '784bada498004820950d0c9b06fcdace',
 'd5c0dc3eef034ef5a24af2b99f05676c',
 '84eb347a065146be985d228456e8e574',
 '61319f845d344da3b3a7ed35d83ab644',
 '93787bf08adc436787e131a4eb40a2ce',
 'b0646e34a17e43d89916c48c5c99235f',
 

In [4]:
from typing import TypedDict
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph

# Graph input/output state
class GraphState(TypedDict):
    question: str
    answer: str
    docs: list

# Create retriever from Qdrant
retriever = qdrant.as_retriever()

# Format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG node function
def rag_node(state: GraphState):
    question = state["question"]
    docs = retriever.get_relevant_documents(question)
    context = format_docs(docs)
    prompt = f"Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}"
    response = llm.invoke(prompt)
    
    return {
        "question": question,
        "answer": response.content,
        "docs": docs
    }

# Build LangGraph
graph = StateGraph(GraphState)
graph.add_node("rag_node", RunnableLambda(rag_node))
graph.set_entry_point("rag_node")
graph.set_finish_point("rag_node")
app = graph.compile()


In [6]:
try:
    response = app.invoke({"question": "tell meThe changing face of customer support acc to pdf"})
    print("Answer:", response["answer"])
    
except Exception as e:
    print("API Error:", e)

Answer: Based on the provided context, it appears that the PDF is discussing the changing face of customer support in the era of the modern consumer. The table of contents suggests that the guide will cover various aspects of customer support, including:

* The importance of involving everyone in the organization in customer support
* Structuring a support organization
* Building a robust product and internal workflows
* Hiring and training a support team
* Measuring support performance with KPIs
* Automating customer support
* Meeting customers on mobile
* Defining high-caliber support

The guide seems to be emphasizing the need for a holistic approach to customer support, where every department and team member plays a role in delivering a great customer experience.
